In [1]:
import os
import cv2
import numpy as np

from torchvision import transforms as T
import torch
import torch.nn as nn

import matplotlib.pyplot as plt

import torchvision

from collections import OrderedDict
import torch.optim as optim

In [2]:
class Apple_Dataset(object):
    def __init__(self, root_dir, img_transform, mask_transform):
        self.root_dir = root_dir
        self.img_transform = img_transform
        self.mask_transform = mask_transform

        # Load all image and mask files, sorting them to ensure they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root_dir, "images"))))
        self.masks = list(sorted(os.listdir(os.path.join(root_dir, "masks"))))

    def __getitem__(self, idx):
        # Load images and masks
        img_path = os.path.join(self.root_dir, "images", self.imgs[idx])
        mask_path = os.path.join(self.root_dir, "masks", self.masks[idx])

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_path, 0)
        
        obj_ids = np.unique(mask)
        num_apples = len(obj_ids) - 1


        # Split the color-encoded masks into a set of binary masks
        target = np.array((mask>0)*255, dtype=np.uint8)
        

        img = self.img_transform(cv2.resize(img, (512,512)) )
        target = self.mask_transform(cv2.resize(target, (512,512)))
        

        # return tensors
        return img, target, num_apples

    def __len__(self):
        return len(self.imgs)

In [3]:
class UNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        super(UNet, self).__init__()

        features = init_features
        self.encoder1 = UNet._block(in_channels, features, name="enc1")
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = UNet._block(features, features * 2, name="enc2")
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = UNet._block(features * 2, features * 4, name="enc3")
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = UNet._block(features * 4, features * 8, name="enc4")
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.bottleneck = UNet._block(features * 8, features * 16, name="bottleneck")

        self.upconv4 = nn.ConvTranspose2d(
            features * 16, features * 8, kernel_size=2, stride=2
        )
        self.decoder4 = UNet._block((features * 8) * 2, features * 8, name="dec4")
        self.upconv3 = nn.ConvTranspose2d(
            features * 8, features * 4, kernel_size=2, stride=2
        )
        self.decoder3 = UNet._block((features * 4) * 2, features * 4, name="dec3")
        self.upconv2 = nn.ConvTranspose2d(
            features * 4, features * 2, kernel_size=2, stride=2
        )
        self.decoder2 = UNet._block((features * 2) * 2, features * 2, name="dec2")
        self.upconv1 = nn.ConvTranspose2d(
            features * 2, features, kernel_size=2, stride=2
        )
        self.decoder1 = UNet._block(features * 2, features, name="dec1")

        self.conv = nn.Conv2d(
            in_channels=features, out_channels=out_channels, kernel_size=1
        )

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))

        bottleneck = self.bottleneck(self.pool4(enc4))

        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)
        return torch.sigmoid(self.conv(dec1))

    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (
                        name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                ]
            )
        )

class DiceLoss(nn.Module):

    def __init__(self):
        super(DiceLoss, self).__init__()
        self.smooth = 1.0

    def forward(self, y_pred, y_true):
        assert y_pred.size() == y_true.size()
        y_pred = y_pred[:, 0].contiguous().view(-1)
        y_true = y_true[:, 0].contiguous().view(-1)
        intersection = (y_pred * y_true).sum()
        dsc = (2. * intersection + self.smooth) / (
            y_pred.sum() + y_true.sum() + self.smooth
        )
        return 1. - dsc

In [4]:
class Utils(object):
    def __init__(self):
        pass
    
    def get_transform(self, ifimg):
        transforms = [T.ToTensor()]
        if ifimg:
            transforms.append(T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)))
        return T.Compose(transforms)
    
    def get_iou_acc(self, y_pred, y_true):
        SMOOTH = 1e-6
        
        y_pred = y_pred[:, 0].contiguous().view(-1)
        y_true = y_true[:, 0].contiguous().view(-1)
        
        target = y_true.cpu().detach().numpy()
        pred = y_pred.cpu().detach().numpy()
        
        pred_binary = np.where(pred > 0.5, 1, 0)
        target_binary = np.where(target > 0.5, 1, 0)

        intersection = (pred_binary & target_binary).sum()
        union = (pred_binary | target_binary).sum()
        
        iou = (intersection + SMOOTH) / (union + SMOOTH)
        
        y_pred_numpy = pred[np.where(pred > 0.5)]

        return iou, y_pred_numpy.mean()  # Or thresholded.mean()
    
    def get_count(self, mask):
        thresh1, bw3 = cv2.threshold(mask, thresh=220, maxval=255,
                                     type=cv2.THRESH_BINARY)  # manually selected threshold, low values to 255
        '''
        Morphological operations
        '''
        # create a structuring element
        kernel = np.ones((3, 3), np.uint8)
        # closing
        closing = cv2.morphologyEx(bw3, cv2.MORPH_CLOSE, kernel, iterations=1)

        # erosion
        kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (5, 5))
        erosion = cv2.erode(closing, kernel, iterations=1)

        # closing
        kernel = np.ones((5, 5), np.uint8)
        closing = cv2.morphologyEx(erosion, cv2.MORPH_CLOSE, kernel, iterations=1)

        # erosion
        kernel = np.ones((5, 5), np.uint8)
        erosion2 = cv2.erode(closing, kernel, iterations=1)

        cnts, _ = cv2.findContours(erosion2.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        # cv2.drawContours(img, cnts, -1, (0, 255, 0), 2)

        cnts_filtered = [c for c in cnts if cv2.contourArea(c) > 10]

        return len(cnts_filtered)


    def my_collate_fn(self, batch):
        images = [each[0] for each in batch]
        targets = [each[1] for each in batch]
        return images, targets


    def generate_bbox(self, img, boxes):
        for (x_min, y_min, x_max, y_max) in boxes:
            cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            
    def get_frcnn_model_instance(self, num_classes):
        # load an instance segmentation model pre-trained pre-trained on COCO
        model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

        # get number of input features for the classifier
        in_features = model.roi_heads.box_predictor.cls_score.in_features
        # replace the pre-trained head with a new one
        model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
        return model

In [5]:
root_path = '../input/minneapple/Dataset/Dataset'
out_folder = '/kaggle/working/apple_counting/'
model_path = '../input/applecountingunet/apple_counting/Models/UNet_18Dec2020_50.pt'

util = Utils()
test_dataset = Apple_Dataset(os.path.join(root_path, 'test'), util.get_transform(True), util.get_transform(False))
data_loader = {'test': torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers=1)}

device = torch.device("cpu" if not torch.cuda.is_available() else "cuda:0")

model = UNet(in_channels=3, out_channels=1)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)

dsc_loss = DiceLoss()
category = 'test'

loss_epoch = []
iou = []
pixel_acc = []
precision_list = []
for i, (images, targets, num_apples) in enumerate(data_loader[category]):
    x = images.to(device)
    y_true = targets.to(device)
    model.eval()
    
    with torch.no_grad():
        y_pred = model(x)
        loss = dsc_loss(y_pred, y_true)
        iou_val, acc_val = util.get_iou_acc(y_pred, y_true)
        
    mask_pred = np.array(y_pred.cpu().detach().data[0].permute(1, 2, 0) * 255, dtype=np.uint8)
    mask_pred = cv2.resize(mask_pred, (720, 1280))
    num_apples_pred = util.get_count(mask_pred.copy())
        
    precision = min(num_apples[0], num_apples_pred)/num_apples[0]

    loss_epoch.append(loss.item())
    iou.append(iou_val.item())
    pixel_acc.append(acc_val)
    precision_list.append(precision)

print ("Dice loss: {}".format(sum(loss_epoch)/len(loss_epoch)))
print ("IoU: {}".format(sum(iou)/len(iou)))
print ("Pixel Acc: {}".format(sum(pixel_acc)/len(pixel_acc)))
print ("Count Accuracy: {}".format(sum(precision_list)/len(precision_list)))

Dice loss: 0.2522872899895284
IoU: 0.6622448005250724
Pixel Acc: 0.9891246072391966
Count Accuracy: 0.6822430491447449
